# Simulator For TRPL Data

In [1]:
import os
os.environ['OMP_NUM_THREADS'] = '1'

import numpy as np
import pandas as pd
import pytensor.tensor as at
from pytensor import *
from pytensor import config
config.allow_gc = False
import matplotlib.pyplot as plt

### Oxford color scheme
base_color = "#002147"
color_scheme = ["#44687d", "#69913b", '#cf7a30', '#be0f34']
highlight_color = "#ac48bf"

line_type_list = ['-', '-.', '--', ':']

centimeters = 1/2.54
fontsize_base = 11
figure_width = 21


WARNING (pytensor.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


# Model

In [463]:
def X_n_maker(d_factor, size, dx, D, Sf, Sb):
    x_size = at.zeros(shape=(at.shape(size),at.shape(size)))
    Xn_1 = at.extra_ops.fill_diagonal_offset(x_size, d_factor, -1)
    
    Xn_2a = at.extra_ops.fill_diagonal_offset(x_size, 1-2.*d_factor, 0)
    Xn_2a1 = at.set_subtensor(Xn_2a[0,0],1-d_factor - (dx/D)*d_factor *Sf)
    Xn_2 = at.set_subtensor(Xn_2a1[-1,-1],1-d_factor - (dx/D)*d_factor *Sb)
    
    Xn_3 = at.extra_ops.fill_diagonal_offset(x_size, d_factor, 1)
    
    return Xn_1 + Xn_2 + Xn_3


### First: Define Rate equations
### First: Define Rate equations
def rate_equations(n_dens, nt, params):            

    k_c, k_deep, k_e, k_rad, k_aug, _, _, _, _ = params
    
    p_dens = n_dens + nt

    R_rad = - k_rad*n_dens*p_dens
    
    dnt_dt = k_c*n_dens - k_e*nt
    R_nr = - k_c*n_dens + k_e*nt - k_deep*n_dens
    
    dn_dt = R_rad + R_nr
    
    return dn_dt, dnt_dt


def Runge_Kutta_R4(n_dens, nt, dt, params):

    RuKu1_n, RuKu1_nt = rate_equations(n_dens, nt, params)
    RuKu2_n, RuKu2_nt = rate_equations(n_dens + RuKu1_n*dt/2, nt + RuKu1_nt*dt/2, params)
    RuKu3_n, RuKu3_nt = rate_equations(n_dens + RuKu2_n*dt/2, nt + RuKu2_nt*dt/2, params)
    RuKu4_n, RuKu4_nt = rate_equations(n_dens + RuKu3_n*dt, nt + RuKu3_nt*dt, params)

    Ruku_n = (RuKu1_n + 2*RuKu2_n + 2*RuKu3_n + RuKu4_n)/6
    Ruku_nt = (RuKu1_nt + 2*RuKu2_nt + 2*RuKu3_nt + RuKu4_nt)/6

    return Ruku_n, Ruku_nt


### Looping over time-domain
def total_recombination_rate(dt_current, n_dens, p_dens, ds, params):

    _, _, _, _, _, _, D, S_f, S_b = params

    # a. Recombination (Runge-Kutta Algorithm)
    nt = p_dens - n_dens
    Ruku_n, Ruku_nt  = Runge_Kutta_R4(n_dens, nt, dt_current, params)
    
    # b. Diffusion
    d_factor = D*dt_current/(2*ds*ds)
    A_n = X_n_maker(-d_factor, n_dens, ds, D, S_f, S_b)
    B_n = X_n_maker(d_factor, n_dens, ds, D, S_f, S_b)

    Bn_dot_n_dens = at.dot(B_n, n_dens) + Ruku_n*dt_current/2
    n_dens_new = at.dot(at.linalg.inv(A_n), Bn_dot_n_dens)

    # c. Physical limits
    n_dens_new = at.switch(at.le(n_dens_new, 0), 0, n_dens_new)
    p_dens_new = n_dens_new + nt + Ruku_nt*dt_current
    p_dens_new = at.switch(at.le(p_dens_new, 0), 0, p_dens_new)
    
    return n_dens_new, p_dens_new

In [464]:
def model_in_pytensor(time, Fluence, thickness, Absorption_coeff, p0, S_front_value, S_back_value, mu_vert, k_rad, k_deep, k_c, k_e):
    
    ## Define the spacial and temporal domains

    x = np.arange(0,thickness,30)
    z_array_np = x*1e-3
    z_array = at.as_tensor_variable(z_array_np)

    ds = at.as_tensor_variable(z_array_np[1]-z_array_np[0])
    dt = at.extra_ops.diff(time)   
    
    # Diffusion Coeffient in cm2 s-1
    limit_mobility = (thickness*1e-7)**2/(at.abs(time[1]-time[0]))/(1.380649e-23*292/1.6021766e-19)  #cm2 (Vs)-1
    Diffusion_coefficient = mu_vert*(1.380649e-23*292/1.6021766e-19)*1e8

    
    # Packing parameters
    params = k_c, k_deep, k_e, k_rad, 0, 0, Diffusion_coefficient, S_front_value, S_back_value
    
    # Initial Charge-Carrier Density
    Absorption_coeff = at.switch(at.ge(mu_vert, limit_mobility/4), 0, Absorption_coeff)
    
    generation = at.exp(-Absorption_coeff*z_array)
    generation_sum = at.sum(((generation[1:] + generation[:-1])/2), axis=0) * ds
    n_0z = Fluence/(generation_sum) * generation
    
    
    result_one_sample, _ = pytensor.scan(fn=total_recombination_rate,
                                            sequences=[dt],
                                            outputs_info=[n_0z, n_0z+p0],
                                            non_sequences=[ds, params], strict=True)    

    n_init = (n_0z).dimshuffle('x',0)
    N_calc = at.concatenate([n_init, result_one_sample[0]], axis=0)
    p_init = (n_0z + p0).dimshuffle('x',0)
    P_calc = at.concatenate([p_init, result_one_sample[1]], axis=0)
 
    ## Turn radiative recombination into PL response    
    Rrad_calc = N_calc * P_calc
    
    PL_calc = at.sum(((Rrad_calc[:,1:] + Rrad_calc[:,:-1])/2), axis=1) * ds
    
    PL_0 = PL_calc[0]

    PL_obs = PL_calc/PL_0    
    return PL_obs, N_calc, P_calc


## Main Part

In [ ]:
# Parameter Definition
Sfront = 0.1 #cm s-1
Sback = 0.1 #cm s-1
Mobility = 100 #cm2 V-1s-1


Fluence = 4.8e11   #cm-2

alpha = 3e5  #cm-1
Thickness = 600   #nm

k_rad = 1e-12   #cm3 s-1
p0 = 0#   #cm-3


k_deep = 1e6   #s-1
k_c = 0
k_e = 0

time = np.logspace(-1,2.8,200) #ns

df_save = pd.DataFrame()
df_save[f'Time'] = time

PL, n_dens, p_dens = model_in_pytensor(time*1e-9, Fluence*1e-8, Thickness, alpha*1e-4, p0*1e-12, Sfront*1e4, Sback*1e4, Mobility, k_rad*1e12, k_deep, k_c, k_e)


In [436]:
n = n_dens.eval() * 1e12
p = p_dens.eval() * 1e12
PL = PL.eval()